# Preliminary Analysis of Divvy Station Status

11/15/23<br>
This notebook explores Divvy's API, to set the stage for subsequent data retrieval and analysis.<br>

<h3>Data mysteries</h3>
<ul>
    <li>figure out why num_ebikes_available does not always match vehicle_type_id = 2 (sometimes greater, sometimes less)</li>
    <li>what do is_renting and is_returning mean?</li>
    <li>how to identify lock vs. dock</li>
</ul>

<h3>Documentation</h3>
Live Station Data from Divvy Site
https://gbfs.divvybikes.com/gbfs/2.3/gbfs.json

# import libraries

In [1]:
import pandas as pd
import requests
import datetime
import pytz

# review vehicle types

In [2]:
url = f"https://gbfs.lyft.com/gbfs/2.3/chi/en/vehicle_types.json?"

#run the query
response = requests.get(url)
data = response.json()
response

<Response [200]>

In [3]:
#navigate through dictionaries to find station data
#data.keys()
data['data'].keys()

dict_keys(['vehicle_types'])

In [4]:
central_tz = pytz.timezone('America/Chicago')
status_timestamp = datetime.datetime.utcfromtimestamp(data['last_updated']).replace(tzinfo=pytz.utc)
status_timestamp = status_timestamp.astimezone(central_tz).strftime('_%Y_%m_%d_%I%M%p')
status_timestamp

'_2023_11_20_1100AM'

In [5]:
#look at all data
data['data']['vehicle_types']

[{'form_factor': 'bicycle',
  'vehicle_type_id': '1',
  'propulsion_type': 'human'},
 {'max_range_meters': 54717.56,
  'form_factor': 'bicycle',
  'vehicle_type_id': '2',
  'propulsion_type': 'electric_assist'},
 {'max_range_meters': 70810.95999999999,
  'form_factor': 'scooter',
  'vehicle_type_id': '3',
  'propulsion_type': 'electric'}]

# review station info

In [6]:
url = f"https://gbfs.lyft.com/gbfs/2.3/chi/en/station_information.json?"

#request data
response = requests.get(url)
data = response.json()
response

<Response [200]>

In [7]:
#navigate through dictionaries to find station data
#data.keys()
data['data'].keys()

dict_keys(['stations'])

In [8]:
#look at the first station
data['data']['stations'][0]

{'lat': 41.916907,
 'name': 'Parkside Ave & Armitage Ave',
 'rental_uris': {'android': 'https://chi.lft.to/lastmile_qr_scan',
  'ios': 'https://chi.lft.to/lastmile_qr_scan'},
 'lon': -87.767173,
 'short_name': '21354',
 'station_id': '1871993715238611622',
 'capacity': 15}

In [9]:
df_station_info = pd.DataFrame(data['data']['stations'], columns=['station_id','name','lat','lon','capacity','short_name'])
df_station_info.head()

,station_id,name,lat,lon,capacity,short_name
0,1871993715238611622,Parkside Ave & Armitage Ave,41.916907,-87.767173,15,21354
1,a3a5de24-a135-11e9-9cda-0a87ae2ba916,Lincoln Ave & Melrose St,41.940600,-87.669435,15,TA1309000042
2,a3a7f0ba-a135-11e9-9cda-0a87ae2ba916,Damen Ave & Madison St,41.881370,-87.674930,15,13134
3,a3a62fc9-a135-11e9-9cda-0a87ae2ba916,Indiana Ave & 26th St,41.845687,-87.622481,15,TA1307000005
4,a3b4399a-a135-11e9-9cda-0a87ae2ba916,Smith Park,41.892048,-87.689397,15,643


# review station status

### review data structures

In [10]:
#run the query
url = f"https://gbfs.lyft.com/gbfs/2.3/chi/en/station_status.json?"
response = requests.get(url)
data = response.json()
response

<Response [200]>

In [11]:
#navigate through dictionaries to find station data
# data.keys()
data['data'].keys()

dict_keys(['stations'])

In [12]:
#look at the first station
data_station_status = data['data']['stations']
data_station_status[0]

{'vehicle_docks_available': [{'vehicle_type_ids': ['1', '2'], 'count': 0}],
 'num_bikes_disabled': 0,
 'num_docks_available': 0,
 'num_docks_disabled': 0,
 'is_installed': 0,
 'is_renting': 0,
 'last_reported': 86400,
 'num_bikes_available': 0,
 'vehicle_types_available': [{'count': 0, 'vehicle_type_id': '1'},
  {'count': 0, 'vehicle_type_id': '2'}],
 'num_ebikes_available': 0,
 'station_id': '1871993715238611622',
 'is_returning': 0}

In [13]:
#read into dataframe and rename columns
df_station_status= pd.DataFrame(data_station_status, columns=
        ['station_id','num_bikes_available','vehicle_docks_available',
         'vehicle_types_available','num_bikes_disabled','num_docks_available','num_ebikes_available'])

### review vehicle type dictionary

In [14]:
#extract vehicle type info
df_station_status['n_classic'] = df_station_status['vehicle_types_available'].apply(
    lambda x: next((item['count'] for item in x if item['vehicle_type_id'] == '1'), 0))
df_station_status['n_electric'] = df_station_status['vehicle_types_available'].apply(
    lambda x: next((item['count'] for item in x if item['vehicle_type_id'] == '2'), 0))
df_station_status['n_scooters'] = df_station_status['vehicle_types_available'].apply(
    lambda x: next((item['count'] for item in x if item['vehicle_type_id'] == '3'), 0))

df_station_status[['station_id','n_classic','n_electric','n_scooters']].head()

,station_id,n_classic,n_electric,n_scooters
0,1871993715238611622,0,0,0
1,a3a5de24-a135-11e9-9cda-0a87ae2ba916,8,2,0
2,a3a7f0ba-a135-11e9-9cda-0a87ae2ba916,6,2,0
3,a3a62fc9-a135-11e9-9cda-0a87ae2ba916,9,1,1
4,a3b4399a-a135-11e9-9cda-0a87ae2ba916,4,1,0


### review dock types dictionary

In [15]:
df_station_status['vehicle_docks_available']

0          [{'vehicle_type_ids': ['1', '2'], 'count': 0}]
1          [{'vehicle_type_ids': ['1', '2'], 'count': 3}]
2       [{'vehicle_type_ids': ['1', '2', '3'], 'count'...
3       [{'vehicle_type_ids': ['1', '2', '3'], 'count'...
4          [{'vehicle_type_ids': ['1', '2'], 'count': 9}]
                              ...                        
1662       [{'vehicle_type_ids': ['1', '2'], 'count': 0}]
1663       [{'vehicle_type_ids': ['1', '2'], 'count': 0}]
1664       [{'vehicle_type_ids': ['1', '2'], 'count': 0}]
1665       [{'vehicle_type_ids': ['1', '2'], 'count': 0}]
1666       [{'vehicle_type_ids': ['1', '2'], 'count': 0}]
Name: vehicle_docks_available, Length: 1667, dtype: object

In [16]:
df_station_status['dock_type_ids'] = df_station_status[
    'vehicle_docks_available'].apply(lambda x: x[0]['vehicle_type_ids'] if x else None)
df_station_status['n_docks'] = df_station_status['vehicle_docks_available'].apply(lambda x: x[0]['count'] if x else None)

In [17]:
df_station_status[['station_id','dock_type_ids','n_docks']]

,station_id,dock_type_ids,n_docks
0,1871993715238611622,"[1, 2]",0
1,a3a5de24-a135-11e9-9cda-0a87ae2ba916,"[1, 2]",3
2,a3a7f0ba-a135-11e9-9cda-0a87ae2ba916,"[1, 2, 3]",7
3,a3a62fc9-a135-11e9-9cda-0a87ae2ba916,"[1, 2, 3]",4
4,a3b4399a-a135-11e9-9cda-0a87ae2ba916,"[1, 2]",9
...,...,...,...
1662,1674190656158837882,"[1, 2]",0
1663,1674190428525570800,"[1, 2]",0
1664,1674190239547009458,"[1, 2]",0
1665,1806749727197839828,"[1, 2]",0


In [18]:
#this is weird, and not so helpful
df_station_status['dock_type_ids'].value_counts()

dock_type_ids
[1, 2]       1427
[1, 2, 3]     240
Name: count, dtype: int64

### review lock-only stations


In [19]:
#review the 3 Humboldt Park stations with no classic bikes- Grand/Hamlin, Central Park/Augusta, Central Park/Ohio)
station_ids = ['1575949519568499428', '1575949510978564830', '1575949515273532128']
df_humboldt3 = df_station_status[df_station_status['station_id'].isin(station_ids)]
df_humboldt3.to_csv("../results/docks_humboldt3.csv")
df_station_status.to_csv("../results/docks_all.csv")

### review station stats

In [20]:
#calculate station data
df_station_status['n_all_bikes']=df_station_status['num_bikes_available']+df_station_status['num_bikes_disabled']
df_station_status['is_no_classic']=(df_station_status['n_classic']==0)
df_station_status['is_no_docks']=(df_station_status['num_docks_available']==0)
df_station_status['is_problem_station']= df_station_status['is_no_classic'] | df_station_status['is_no_docks']
df_station_status['station_id_len']=df_station_status['station_id'].str.len()

df_station_status[['station_id','n_all_bikes','is_no_classic','is_no_docks','is_problem_station','station_id_len']].head()

,station_id,n_all_bikes,is_no_classic,is_no_docks,is_problem_station,station_id_len
0,1871993715238611622,0,True,True,True,19
1,a3a5de24-a135-11e9-9cda-0a87ae2ba916,12,False,False,False,36
2,a3a7f0ba-a135-11e9-9cda-0a87ae2ba916,8,False,False,False,36
3,a3a62fc9-a135-11e9-9cda-0a87ae2ba916,10,False,False,False,36
4,a3b4399a-a135-11e9-9cda-0a87ae2ba916,6,False,False,False,36


# Review Free Bikes
this looks like only stray e-bikes and scooters with nothing re: classic bikes

In [22]:
url = f"https://gbfs.lyft.com/gbfs/2.3/chi/en/free_bike_status.json?"
response = requests.get(url)
data = response.json()
response

<Response [200]>

In [32]:
data['data']['bikes']

[{'lat': 41.80938783333333,
  'bike_id': 'e25864fef418f4503f93a46fc88d0878',
  'is_disabled': 0,
  'vehicle_type_id': '2',
  'is_reserved': 0,
  'rental_uris': {'android': 'https://chi.lft.to/lastmile_qr_scan',
   'ios': 'https://chi.lft.to/lastmile_qr_scan'},
  'lon': -87.6200885,
  'current_range_meters': 14323.161600000001},
 {'lat': 41.960619,
  'bike_id': 'ee0953947ea06873deb6e45884c91e06',
  'is_disabled': 0,
  'vehicle_type_id': '2',
  'is_reserved': 0,
  'rental_uris': {'android': 'https://chi.lft.to/lastmile_qr_scan',
   'ios': 'https://chi.lft.to/lastmile_qr_scan'},
  'lon': -87.76487366666667,
  'current_range_meters': 40072.6656},
 {'lat': 41.88539183333334,
  'bike_id': 'd3c9b030d3ef88b936cf34df1bd6ed1c',
  'is_disabled': 0,
  'vehicle_type_id': '2',
  'is_reserved': 0,
  'rental_uris': {'android': 'https://chi.lft.to/lastmile_qr_scan',
   'ios': 'https://chi.lft.to/lastmile_qr_scan'},
  'lon': -87.726707,
  'current_range_meters': 32830.6176},
 {'lat': 41.87773116666666,


In [36]:
df_bikes=pd.DataFrame(data['data']['bikes'])
df_bikes.head()

,lat,bike_id,is_disabled,vehicle_type_id,is_reserved,rental_uris,lon,current_range_meters
0,41.809388,e25864fef418f4503f93a46fc88d0878,0,2,0,{'android': 'https://chi.lft.to/lastmile_qr_sc...,-87.620088,14323.1616
1,41.960619,ee0953947ea06873deb6e45884c91e06,0,2,0,{'android': 'https://chi.lft.to/lastmile_qr_sc...,-87.764874,40072.6656
2,41.885392,d3c9b030d3ef88b936cf34df1bd6ed1c,0,2,0,{'android': 'https://chi.lft.to/lastmile_qr_sc...,-87.726707,32830.6176
3,41.877731,032f67e8a6723c566d51a9360c9744bd,0,2,0,{'android': 'https://chi.lft.to/lastmile_qr_sc...,-87.634248,17863.7184
4,41.997115,41cc34956f725692fe67cd6528556944,0,2,0,{'android': 'https://chi.lft.to/lastmile_qr_sc...,-87.670155,14805.9648


In [38]:
df_bikes['vehicle_type_id'].value_counts()

vehicle_type_id
2    1705
3     343
Name: count, dtype: int64

# Merge Station Info
to assist with further exploration, esp. public racks and dock types

In [116]:
df_stations = pd.merge(df_station_status, df_station_info, on='station_id')
df_stations.head()

,station_id,num_bikes_available,vehicle_docks_available,vehicle_types_available,num_bikes_disabled,num_docks_available,num_ebikes_available,n_classic,n_electric,n_scooters,...,n_all_bikes,is_no_classic,is_no_docks,is_problem_station,station_id_len,name,lat,lon,capacity,short_name
0,a3a565b1-a135-11e9-9cda-0a87ae2ba916,6,"[{'vehicle_type_ids': ['1', '2', '3'], 'count'...","[{'count': 5, 'vehicle_type_id': '1'}, {'count...",0,0,1,5,1,0,...,6,False,True,True,36,Halsted St & Polk St,41.871840,-87.646640,19,TA1307000121
1,a3b29fae-a135-11e9-9cda-0a87ae2ba916,2,"[{'vehicle_type_ids': ['1', '2'], 'count': 9}]","[{'count': 2, 'vehicle_type_id': '1'}, {'count...",0,9,0,2,0,0,...,2,False,False,False,36,Stony Island Ave & 82nd St,41.746559,-87.586005,11,583
2,4843d153-c725-4230-a4c7-402da6a62143,11,"[{'vehicle_type_ids': ['1', '2'], 'count': 2}]","[{'count': 10, 'vehicle_type_id': '1'}, {'coun...",0,2,1,10,1,0,...,11,False,False,False,36,Lincoln Ave & Roscoe St*,41.943350,-87.670668,15,chargingstx5
3,a3afb401-a135-11e9-9cda-0a87ae2ba916,4,"[{'vehicle_type_ids': ['1', '2'], 'count': 11}]","[{'count': 4, 'vehicle_type_id': '1'}, {'count...",0,11,0,4,0,0,...,4,False,False,False,36,Monticello Ave & Irving Park Rd,41.954005,-87.719128,15,KA1504000139
4,a3acea44-a135-11e9-9cda-0a87ae2ba916,4,"[{'vehicle_type_ids': ['1', '2'], 'count': 6}]","[{'count': 3, 'vehicle_type_id': '1'}, {'count...",1,6,1,3,1,0,...,5,False,False,False,36,Calumet Ave & 71st St,41.765508,-87.616918,11,15599


In [117]:
#identify and remove public racks
df_stations['is_public_rack']=df_stations['name'].str[:11]=='Public Rack'

In [121]:
df_station_maps = df_stations[['station_id','name','dock_type_ids','is_public_rack','capacity','lat','lon']]
df_station_maps

,station_id,name,dock_type_ids,is_public_rack,capacity,lat,lon
0,a3a565b1-a135-11e9-9cda-0a87ae2ba916,Halsted St & Polk St,"[1, 2, 3]",False,19,41.871840,-87.646640
1,a3b29fae-a135-11e9-9cda-0a87ae2ba916,Stony Island Ave & 82nd St,"[1, 2]",False,11,41.746559,-87.586005
2,4843d153-c725-4230-a4c7-402da6a62143,Lincoln Ave & Roscoe St*,"[1, 2]",False,15,41.943350,-87.670668
3,a3afb401-a135-11e9-9cda-0a87ae2ba916,Monticello Ave & Irving Park Rd,"[1, 2]",False,15,41.954005,-87.719128
4,a3acea44-a135-11e9-9cda-0a87ae2ba916,Calumet Ave & 71st St,"[1, 2]",False,11,41.765508,-87.616918
...,...,...,...,...,...,...,...
1661,1715823822654071770,Public Rack - Oakley Ave & 50th Pl,"[1, 2]",True,1,41.801930,-87.680903
1662,1563698701206292480,Lamon Ave & Belmont Ave,"[1, 2]",False,9,41.939011,-87.749283
1663,1674190591734328324,Public Rack - Racine Ave & 76th,"[1, 2]",True,2,41.755786,-87.654054
1664,1806749735787774444,Public Rack - Central Ave & Lawrence Ave,"[1, 2]",True,4,41.967864,-87.767593


In [122]:
df_station_maps.to_csv('../results/station_map.csv')